In [ ]:
import pandas as pd
import geopandas as gpd
import gtfs_functions

In [ ]:
import matplotlib as mpl
mpl.rcParams["figure.dpi"] = 300

In [ ]:
hennepin_df = gpd.read_file("hennepin-county-property-map-cleaned.geojson")

In [ ]:
ramsey_df = gpd.read_file("ramsey-county-property-map-cleaned.geojson")

In [ ]:
anoka_df = gpd.read_file("anoka-county-property-map-cleaned.geojson")

In [ ]:
feed = gtfs_functions.Feed("gtfs.zip")

In [ ]:
routes = feed.routes[feed.routes.route_id.isin(["901", "902", "903", "904", "921", "923", "924", "888"])]
routes

In [ ]:
stop_ids = feed.stop_times[feed.stop_times.route_id.isin(routes.route_id)].stop_id.unique()

In [ ]:
stops = feed.stops[feed.stops.stop_id.isin(stop_ids)]
stops

In [ ]:
WKT = """PROJCS["NAD_1983_HARN_Adj_MN_Hennepin_Feet",
    GEOGCS["GCS_NAD_1983_HARN_Adj_MN_Hennepin",
        DATUM["D_NAD_1983_HARN_Adj_MN_Hennepin",
            SPHEROID["S_GRS_1980_Adj_MN_Hennepin",6378418.941,298.257222100883,
                AUTHORITY["ESRI","107726"]],
            AUTHORITY["ESRI","106726"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["ESRI","104726"]],
    PROJECTION["Lambert_Conformal_Conic_2SP"],
    PARAMETER["latitude_of_origin",44.7911111111111],
    PARAMETER["central_meridian",-93.3833333333333],
    PARAMETER["standard_parallel_1",44.8833333333333],
    PARAMETER["standard_parallel_2",45.1333333333333],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",100000],
    UNIT["US survey foot",0.304800609601219,
        AUTHORITY["EPSG","9003"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["ESRI","103734"]]
"""

In [ ]:
hennepin_projected = hennepin_df.to_crs(crs=WKT)
ramsey_projected = ramsey_df.to_crs(crs=WKT)
anoka_projected = anoka_df.to_crs(crs=WKT)
property_df = pd.concat([
    gpd.GeoDataFrame(geometry=hennepin_projected.geometry),
    gpd.GeoDataFrame(geometry=ramsey_projected.geometry),
    gpd.GeoDataFrame(geometry=anoka_projected.geometry),
])

In [ ]:
stops_projected = stops.to_crs(crs=WKT)

In [ ]:
buffered_stops = stops_projected.buffer(5280 / 2).unary_union
buffered_stops

In [ ]:
property_df["near_stop"] = property_df.geometry.make_valid().intersects(buffered_stops)

In [ ]:
property_df.near_stop.count(), property_df.near_stop.sum()

In [ ]:
property_df.plot("near_stop", categorical=True)